In [ ]:
! pip install xlsxwriter

In [26]:
import pandas as pd
import numpy as np
import xlsxwriter 

summary_data = pd.DataFrame({
    'Team': ['Alpha', 'Beta', 'Gamma', 'Delta'],
    'Sales_Total': [4500, 5800, 3100, 6200],
    'Expeditions': [10, 12, 8, 15],
    'Success_Rate': [0.90, 0.85, 0.75, 0.95]
})

In [27]:
print("--- The Unstyled Summary Data ---")
print(summary_data)
print("-" * 60)

--- The Unstyled Summary Data ---
    Team  Sales_Total  Expeditions  Success_Rate
0  Alpha         4500           10          0.90
1   Beta         5800           12          0.85
2  Gamma         3100            8          0.75
3  Delta         6200           15          0.95
------------------------------------------------------------


In [29]:
def starify(val):
    """Converts a success rate (0-1) into a string of up to 5 stars."""
    num_stars = int(val * 5) # Scale to 5 stars, truncate decimals
    return '★' * num_stars

In [30]:
summary_data['Success_Rating_Visual'] = summary_data['Success_Rate'].apply(starify)

print("\n--- Summary Data with New Yellow Emoji Visual Column ---")
print(summary_data)
print("-" * 60)


--- Summary Data with New Yellow Emoji Visual Column ---
    Team  Sales_Total  Expeditions  Success_Rate Success_Rating_Visual
0  Alpha         4500           10          0.90                  ★★★★
1   Beta         5800           12          0.85                  ★★★★
2  Gamma         3100            8          0.75                   ★★★
3  Delta         6200           15          0.95                  ★★★★
------------------------------------------------------------


In [34]:
try:
  with pd.ExcelWriter('styled_report.xlsx', engine='xlsxwriter') as writer:    
    summary_data.to_excel(writer, sheet_name='Report', index=False)

    # 1) Write the raw data (including the emoji column) to Excel
    workbook = writer.book
    worksheet = writer.sheets['Report']
    num_rows = len(summary_data)

    # 2) Highlight the max Sales_Total
    max_idx = summary_data['Sales_Total'].idxmax()
    fmt_max = workbook.add_format({'bg_color': '#C6EFCE', 'bold': True})
    worksheet.write(max_idx + 1, 1, summary_data.loc[max_idx, 'Sales_Total'],
                                                     fmt_max)

    # 3) Add a data bar to Sales_Total    
    worksheet.conditional_format(f'B2:B{num_rows+1}', {'type': 'data_bar'})

    # 4) Apply number formats    
    money_fmt  = workbook.add_format({'num_format': '$#,##0'})
    percent_fmt = workbook.add_format({'num_format': '0.0%'})
    worksheet.set_column('B:B', None, money_fmt)  # Sales_Total
    worksheet.set_column('D:D', None, percent_fmt) # Success_Rate

    # 5) Overwrite the 'Success_Rating_Visual' column with colored ★ stars
    #  This replaces the black emoji with gilted stars in gold.
    star_fmt = workbook.add_format({'font_color': '#FFD700', 'bold': True})    
    for row, rate in enumerate(summary_data['Success_Rate'], start=1):
      stars = '★' * int(rate * 5)
      worksheet.write_string(row, 4, stars, star_fmt)

  print("✅ Excel report created: 'styled_report.xlsx' — open it to see colored stars, data bars, and highlights!")
except Exception as e: print(f"An error occurred during Excel export: {e}")

✅ Excel report created: 'styled_report.xlsx' — open it to see colored stars, data bars, and highlights!
